# Plot contour line Y band resolution vs (dPWV,PWV)

- author Sylvie Dagoret-Campagne
- affiliation IJCLab
- creation date : 2025/11/18
- last update : 2025/11/18
- last update : 2025/11/18 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.cm as cm
from matplotlib.lines import Line2D


from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd
import h5py

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [ ]:
from scipy.interpolate import splprep, splev

## Functions

In [ ]:
def resample_curve(x, y, spacing):
    """Return a new curve with points spaced approximately 'spacing' apart."""
    # parametrize original curve
    tck, u = splprep([x, y], s=0)
    
    # coordinates length
    pts = np.vstack([x, y]).T
    d = np.sqrt(((pts[1:] - pts[:-1])**2).sum(axis=1))
    s_total = d.sum()
    
    # new parameterization
    u_new = np.linspace(0, 1, int(s_total/spacing))
    x_new, y_new = splev(u_new, tck)
    return np.array([x_new, y_new]).T

## Configuration

In [ ]:
!ls data/mag_vs_zdpwvzpwv 

In [ ]:
data_path = "data/mag_vs_zdpwvzpwv"

In [ ]:
all_airmass = np.array([1.2,1.5,2.0]) 

In [ ]:
Delta_PWV = 0.5
Delta_PWV_num = int(Delta_PWV*100.)
#all_PWV = np.arange(Delta_PWV,20.,Delta_PWV)
#print("PWV values",all_PWV) 

In [ ]:
#all_DPWV = np.arange(2.,0.,-0.01)
Delta_dPWV = 0.02
Delta_dPWV_num = int(Delta_dPWV*1000.)
#all_DPWV = np.arange(Delta_dPWV,1.,Delta_dPWV)
#print(all_DPWV)

## Read file

In [ ]:
idx_am = 0

In [ ]:
am = all_airmass[idx_am] 
am_num = int(am*10.)

In [ ]:
filename_input =  f"MagResolutionFromPWVvsPWV_am{am_num}_DPWV{Delta_PWV_num}_DdPWV{Delta_dPWV_num}.h5"
fullfilename_input = os.path.join(data_path,filename_input)

In [ ]:
with h5py.File(fullfilename_input, "r") as f:
    print(list(f.keys()))

In [ ]:
with h5py.File(fullfilename_input, "r") as f:
    zPWV   = f["zPWV"][:]
    zdPWV  = f["zdPWV"][:]
    tresY  = f["tresY"][:]
    tresZY = f["tresZY"][:]

## Plot

In [ ]:
x = zPWV               # shape (nx,)
y = zdPWV           # shape (ny,)
Z = tresY              # shape (ny, nx)

X, Y = np.meshgrid(x, y)

plt.figure(figsize=(10,8))
pcm = plt.pcolormesh(X, Y, Z, shading='auto',cmap='magma')   # shading='auto' avoids dimension issues

cs = plt.contour(X, Y, Z, levels= np.array([0.5,1.,5.,10.,15.,20.,25.,30.]) ,colors='w', linewidths=1.0)
plt.clabel(cs, inline=True, fontsize=8)

plt.colorbar(pcm, label='$\\sigma(m_y)$ (mmag)')
plt.xlabel('airmass  x PWV (mm)')
plt.ylabel('airmass  x ΔPWV (mm)')
plt.title(f'Magnitude resolution (Y band, airmass = {am})')

ax = plt.gca()
ax.minorticks_on()
#ax.set_xticks(np.arange(zPWV.min(), zPWV.max(), 0.1))   # 1 mm steps
#ax.set_yticks(np.arange(all_DPWV.min(), all_DPWV.max()+0.01, -0.1))  # 0.1 mm steps

plt.show()

In [ ]:
plt.figure(figsize=(10,8))

im = plt.imshow(Z, origin='lower',
           extent=[x.min(), x.max(), y.min(), y.max()],
           aspect='auto',cmap='magma')

cs = plt.contour(X, Y, Z, levels= np.array([0.5,1.,5.,10.,15.,20.,25.,30.]) ,colors='w', linewidths=1.0)
plt.clabel(cs, inline=True, fontsize=8)

plt.colorbar(im,label='$\\sigma(m_y)$ (mmag)')
plt.xlabel('airmass x PWV (mm)')
plt.ylabel('airmass x ΔPWV (mm)')
plt.title(f'Magnitude resolution (Y band, airmass = {am})')

ax = plt.gca()
ax.minorticks_on()
#ax.set_xticks(np.arange(zPWV.min(), zPWV.max()+1, 1))   # 1 mm steps
#ax.set_yticks(np.arange(all_DPWV.min(), all_DPWV.max()+0.01, -0.1))  # 0.1 mm steps

plt.show()

In [ ]:
plt.figure(figsize=(10,8))

plt.imshow(Z, origin='lower', extent=[x.min(),x.max(),y.min(),y.max()],
           aspect='auto', cmap='magma',alpha=0.1)

for level, seglist in zip(cs.levels, cs.allsegs):
    for curve in seglist:
        plt.plot(curve[:,0], curve[:,1], '-', linewidth=1.0, label=f"Level {level}")
plt.xlabel('airmass x PWV (mm)')
plt.ylabel('airmass x ΔPWV (mm)')
plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
plt.title(f'Magnitude resolution (Y band, airmass = {am})')

In [ ]:
plt.figure(figsize=(10,8))


for level, seglist in zip(cs.levels, cs.allsegs):
    for curve in seglist:
        plt.plot(curve[:,0], curve[:,1]/level, '-', linewidth=1.0, label=f"Level {level}")
plt.xlabel('airmass x PWV (mm)')
plt.ylabel('airmass x ΔPWV (mm)')
plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
plt.title(f'Magnitude resolution (Y band, airmass = {am})')

## Smooth the contour lines

In [ ]:
spacing = 0.1  # choose your spacing in same units as X,Y grid

resampled_contours = {}  # dict[level] = [array_of_points_per_curve]

for level, seglist in zip(cs.levels, cs.allsegs):
    resampled_contours[level] = []
    for curve in seglist:
        xc = curve[:,0]
        yc = curve[:,1]
        if len(xc) < 4:   # or < k+1
            continue
        new_curve = resample_curve(xc, yc, spacing)
        resampled_contours[level].append(new_curve)


In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(Z, origin='lower', extent=[x.min(),x.max(),y.min(),y.max()],
           aspect='auto', cmap='magma',alpha=0.1)

for level, curves in resampled_contours.items():
    for c in curves:
        plt.plot(c[:,0], c[:,1], '-', linewidth=0.6,label=f"Level {level}")

plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5))
plt.title("Resampled contour curves")
plt.show()


## Colored

In [ ]:
levels = cs.levels
cmap = plt.cm.coolwarm
norm = colors.Normalize(vmin=min(levels), vmax=max(levels))

In [ ]:
plt.figure(figsize=(10,8))

# --- Background image ---
plt.imshow(Z, origin='lower',
           extent=[x.min(),x.max(),y.min(),y.max()],
           aspect='auto', cmap='magma', alpha=0.1)

# --- Colormap for contour lines ---
levels = cs.levels
cmap = plt.cm.coolwarm            # dark blue → red
norm = colors.Normalize(vmin=min(levels), vmax=max(levels))

# --- Plot contour lines manually ---
for level, seglist in zip(cs.levels, cs.allsegs):
    color = cmap(norm(level))
    for curve in seglist:
        plt.plot(curve[:,0], curve[:,1],
                 '-', linewidth=2.0, color=color)

# --- Legend: 1 entry per level ---
legend_elements = [
    Line2D([0], [0], color=cmap(norm(level)), lw=2,
           label=f"Level {level:g}")
    for level in cs.levels
]
plt.legend(handles=legend_elements,
           loc='center left', bbox_to_anchor=(1.05, 0.5),
           title="Contour levels")

plt.xlabel('airmass × PWV (mm)')
plt.ylabel('airmass × ΔPWV (mm)')
plt.title(f'Magnitude resolution (Y band, airmass = {am})')

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10,8))

# --- Background image ---
#plt.imshow(Z, origin='lower',
#           extent=[x.min(),x.max(),y.min(),y.max()],
#           aspect='auto', cmap='magma', alpha=0.1)

# --- Colormap for contour lines ---
levels = cs.levels
cmap = plt.cm.coolwarm            # dark blue → red
norm = colors.Normalize(vmin=min(levels), vmax=max(levels))

# --- Plot contour lines manually ---
for level, seglist in zip(cs.levels, cs.allsegs):
    color = cmap(norm(level))
    for curve in seglist:
        plt.plot(curve[:,0], curve[:,1]/level,
                 '-', linewidth=2.0, color=color)

# --- Legend: 1 entry per level ---
legend_elements = [
    Line2D([0], [0], color=cmap(norm(level)), lw=2,
           label=f"Level {level:g}")
    for level in cs.levels
]
plt.legend(handles=legend_elements,
           loc='center left', bbox_to_anchor=(1.05, 0.5),
           title="Contour levels")

plt.xlabel('airmass × PWV (mm)')
plt.ylabel('airmass × ΔPWV (mm)')
plt.title(f'Magnitude resolution (Y band, airmass = {am})')

plt.tight_layout()
plt.show()
